In [1]:
#profitableHospital-project

In [2]:
#dependencies
import pandas as pd
import requests
import json
import scipy.stats as st
from scipy.stats import linregress
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVC 
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
import tensorflow
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
import numpy as np
import joblib

In [3]:
# ... File Imports

In [4]:
#2017 Hospital Data from Centers for Medicare Services
df = pd.read_csv("Medicare_Hospital_Cost_Report_PUF_2016.csv")
df.head(2)

,Provider CCN,Hospital Name,Street Address,City,State Code,Zip Code,County,Medicare CBSA Number,Rural Versus Urban,CCN Facility Type,...,Total Other Income,Total Income,Total Other Expenses,Net Income,Cost To Charge Ratio,Net Revenue from Medicaid,Medicaid Charges,Net Revenue from Stand-Alone SCHIP,Stand-Alone SCHIP Charges,rpt_rec_num
0,310050,SAINT CLARES HOSPITAL- DENVILLE,25 POCONO ROAD,DENVILLE,NJ,07834,MORRIS,35644.0,U,STH,...,NaN,8399912.0,NaN,8399912.0,0.226854,2010908.0,14023081.0,NaN,NaN,577882
1,670110,FIRST TEXAS HOSPITAL CARROLLTON,1407 E TRINITY MILLS ROAD,CARROLLTON,TX,75006,NaN,NaN,NaN,STH,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,581059


In [5]:
#Quick Check
df['Provider CCN'].count()

6211

In [6]:
#Certificate of Need Status by State File
conTable = pd.read_csv("certificateofneedstatesTable.csv")
conTable.head(2)

,State Code,CON
0,AK,1
1,AL,1


In [7]:
#String to Numeric Conversion Table for Urban v. Rural designation
urbanTable = pd.read_csv("ruralversusurbanTable.csv")
urbanTable.head()

,Rural Versus Urban,Urban
0,U,1
1,R,0


In [8]:
#Lattitude & Longitude for the 2017 hospital listing
geoTable = pd.read_csv("ccn2geocode.csv")
geoTable.head(2)

,Provider CCN,lat,lng
0,10001,31.215884,-85.363008
1,10005,34.222103,-86.160908


In [9]:
# ... Appending of source files into one main working file

In [10]:
mainFile1 = df

In [11]:
#Append a lattitude & longitude
mainFile1 = pd.merge(mainFile1, geoTable, how= "left", on=['Provider CCN'])
#mainFile1.head(2)

In [12]:
#Quick Check
mainFile1['Provider CCN'].count()

6211

In [13]:
#Append certificate of need indicator
mainFile1 = pd.merge(mainFile1, conTable, how= "left", on=['State Code'])
#mainFile1.head()

In [14]:
#Quick Check
mainFile1['Provider CCN'].count()

6211

In [15]:
#Append a numeric representation of rural v. urban
mainFile1 = pd.merge(mainFile1, urbanTable, how= "left", on=['Rural Versus Urban'])

In [16]:
#Quick Check
mainFile1['Provider CCN'].count()

6211

In [17]:
mainFile1.head(2)

,Provider CCN,Hospital Name,Street Address,City,State Code,Zip Code,County,Medicare CBSA Number,Rural Versus Urban,CCN Facility Type,...,Cost To Charge Ratio,Net Revenue from Medicaid,Medicaid Charges,Net Revenue from Stand-Alone SCHIP,Stand-Alone SCHIP Charges,rpt_rec_num,lat,lng,CON,Urban
0,310050,SAINT CLARES HOSPITAL- DENVILLE,25 POCONO ROAD,DENVILLE,NJ,07834,MORRIS,35644.0,U,STH,...,0.226854,2010908.0,14023081.0,NaN,NaN,577882,40.894920,-74.465444,1.0,1.0
1,670110,FIRST TEXAS HOSPITAL CARROLLTON,1407 E TRINITY MILLS ROAD,CARROLLTON,TX,75006,NaN,NaN,NaN,STH,...,NaN,NaN,NaN,NaN,NaN,581059,32.985609,-96.898518,0.0,NaN


In [18]:
mainFile1.to_csv("main1_2016.csv", index=False, header=True)

In [19]:
# ... Add calculated fields

In [20]:
mainFile2 = mainFile1

In [21]:
#labor Expense
lbrexpense = mainFile2["Wage-Related Costs (Core)"] + mainFile2["Total Salaries (adjusted)"]
mainFile2["Labor Exp"] = lbrexpense
#wagepercent.head()

In [22]:
#Append calculated field inpatient percent
inpatient = mainFile2["Inpatient Revenue"]/mainFile2["Gross Revenue"]
mainFile2["Inpatient %"] = inpatient
#mainFile2.head()

In [23]:
#Append calculated field outpatient percent
outpatient = mainFile2["Outpatient Revenue"]/mainFile2["Gross Revenue"]
mainFile2["Outpatient %"] = outpatient
#mainFile2.head()

In [24]:
#Append calculated field charity percent
charity = mainFile2["Cost of Charity Care"]/mainFile2["Net Patient Revenue"]
mainFile2["Charity %"] = charity
#mainFile2.head()

In [25]:
#Append calculated field uncompensated percent
uncompensated = mainFile2["Cost of Uncompensated Care"]/mainFile2["Net Patient Revenue"]
mainFile2["Uncompensated %"] = charity
#mainFile2.head()

In [26]:
#Append calculated field revenue per discharge
revdischarge = mainFile2["Net Patient Revenue"]/mainFile2["Total Discharges (V + XVIII + XIX + Unknown)"]
mainFile2["Rev/Discharge"] = revdischarge
#mainFile2.head()

In [27]:
#Append calculated field revenue per bed
revbed = mainFile2["Net Patient Revenue"]/mainFile2["Number of Beds"]
mainFile2["Rev Per Bed"] = revbed
#mainFile2.head()

In [28]:
#Append calculated field revenue per bed day
revbed = mainFile2["Net Patient Revenue"]/mainFile2["Total Bed Days Available"]
mainFile2["Rev Per Bed Days"] = revbed
#mainFile2.head()

In [29]:
#Append calculated field revs per fte
revfte = mainFile2["Net Patient Revenue"]/mainFile2["FTE - Employees on Payroll"]
mainFile2["Rev Per FTE"] = revfte
#mainFile2.head()

In [30]:
#Append calculated field discharges per bed
volbed = mainFile2["Total Discharges (V + XVIII + XIX + Unknown)"]/mainFile2["Number of Beds"]
mainFile2["Discharges Per Bed"] = volbed
#mainFile2.head()

In [31]:
#Append calculated field discharges per bed day
voldays = mainFile2["Total Discharges (V + XVIII + XIX + Unknown)"]/mainFile2["Total Bed Days Available"]
mainFile2["Discharges Per Bed Days"] = voldays
#mainFile2.head()

In [32]:
#Append calculated field discharges per bed
volfte = mainFile2["Total Discharges (V + XVIII + XIX + Unknown)"]/mainFile2["FTE - Employees on Payroll"]
mainFile2["Discharge/FTE"] = volbed
#mainFile2.head()

In [33]:
#labor percent
wagepercent = mainFile2["Wage-Related Costs (Core)"] + mainFile2["Total Salaries (adjusted)"]
wagepercent = wagepercent / mainFile2["Net Patient Revenue"]
mainFile2["Labor %"] = wagepercent
#wagepercent.head()

In [34]:
#Overhead percent
overheadpercent = mainFile2["Overhead Non-Salary Costs"] / mainFile2["Net Patient Revenue"]
mainFile2["Overhead %"] = overheadpercent
#overheadpercent.head()

In [35]:
#Depreciation percent
deprpercent = mainFile2["Depreciation Cost"] / mainFile2["Net Patient Revenue"]
mainFile2["Depreciation %"] = deprpercent
#overheadpercent.head()

In [36]:
#labor Per FTE
wagefte = mainFile2["Wage-Related Costs (Core)"] + mainFile2["Total Salaries (adjusted)"]
wagefte = wagefte / mainFile2["FTE - Employees on Payroll"]
mainFile2["LbrExp/FTE"] = wagefte
#wagepercent.head()

In [37]:
#Overhead Expense Per Bed
overbed = mainFile2["Overhead Non-Salary Costs"] / mainFile2["Number of Beds"]
mainFile2["Overhead Exp Per Bed"] = overbed
#wagepercent.head()

In [38]:
#Append calculated field net income percent to file
profit = mainFile2["Net Income"]/mainFile2["Net Patient Revenue"]
mainFile2["Net Income %"] = profit
#mainFile2.head()

In [39]:
# Review net income percent for outlier reduction approach
netinc = mainFile2["Net Income %"]
#netinc.head()

In [40]:
# Box plot and outlier calculations for outlier approach
netinc2 = netinc.dropna(how='any')
chartdata = netinc2
# fig1, ax1 = plt.subplots()
# ax1.set_title('Net Income Percent')
# ax1.set_ylabel('Net Income Percent')
# ax1.boxplot(chartdata)
# plt.show()

In [41]:
#Quartile calculations
quartiles = chartdata.quantile([.25,.5,.75])
lowerq = quartiles[0.25]
upperq = quartiles[0.75]
iqr = upperq-lowerq

print(f"The lower quartile of profit percent is: {lowerq}")
print(f"The upper quartile of profit percent is: {upperq}")
print(f"The interquartile range of profit percent is: {iqr}")
print(f"The the median of profit percent is: {quartiles[0.5]} ")

lower_bound = lowerq - (1.5*iqr)
upper_bound = upperq + (1.5*iqr)
print(f"Values below {lower_bound} could be outliers.")
print(f"Values above {upper_bound} could be outliers.")

The lower quartile of profit percent is: -0.02788460161762867
The upper quartile of profit percent is: 0.11236402337209221
The interquartile range of profit percent is: 0.14024862498972088
The the median of profit percent is: 0.040076138202432846 
Values below -0.23825753910220998 could be outliers.
Values above 0.3227369608566735 could be outliers.


In [42]:
#Create a bronze, silver, gold score for clasifying net income performance
bins = [-100, -0.030605, 0.113797, 100]
group_names = ["3", "2", "1"]

In [43]:
#Quintile attempt
# quartiles = chartdata.quantile([.2,.5,.8])
# lowerq = quartiles[0.2]
# upperq = quartiles[0.8]
# iqr = upperq-lowerq

# print(f"The lower quartile of profit percent is: {lowerq}")
# print(f"The upper quartile of profit percent is: {upperq}")
# print(f"The interquartile range of profit percent is: {iqr}")
# print(f"The the median of profit percent is: {quartiles[0.5]} ")

# lower_bound = lowerq - (1.5*iqr)
# upper_bound = upperq + (1.5*iqr)
# print(f"Values below {lower_bound} could be outliers.")
# print(f"Values above {upper_bound} could be outliers.")

In [44]:
#Create a bronze, silver, gold score for clasifying net income performance
# bins = [-100, -0.0542, .03804, 0.13654, 100]
# group_names = ["4","3", "2", "1"]

In [45]:
#Add a bronze, silver, gold scoring system to the dataset
mainFile2["Net Income Score"] = pd.cut(mainFile2["Net Income %"], bins, labels=group_names, include_lowest=True)
#mainFile2.head()

In [46]:
#Quick check on distribution of bronze, silver and gold
mainFile2['Net Income Score'].value_counts()

2    3036
1    1458
3    1446
Name: Net Income Score, dtype: int64

In [47]:
#Quick Check
mainFile2['Provider CCN'].count()

6211

In [48]:
#mainFile2.to_csv("main2_2016.csv", index=False, header=True)

In [49]:
# ..... File targeting

In [50]:
mainFile3 = mainFile2
#mainFile3['Provider CCN'].count()

In [51]:
#Removes invalid row
mainFile3 = mainFile3.loc[mainFile3['Hospital Name'] != '20994']

In [52]:
#Targets dataset on Colorad0
#mainFile3 = mainFile3.loc[df['State Code'] == 'CO']

In [53]:
#Removes state
mainFile3 = mainFile3.loc[mainFile3['State Code'] != 'PR']

In [54]:
#Removes state
mainFile3 = mainFile3.loc[mainFile3['State Code'] != 'GU']

In [55]:
#Removes state
mainFile3 = mainFile3.loc[mainFile3['State Code'] != 'VI']

In [56]:
#Removes state
mainFile3 = mainFile3.loc[mainFile3['State Code'] != 'MP']

In [57]:
#Removes state
mainFile3 = mainFile3.loc[mainFile3['State Code'] != 'HI']

In [58]:
#Removes state
mainFile3 = mainFile3.loc[mainFile3['State Code'] != 'AK']

In [59]:
#Quick Check ... Lose 117 to state code reduction
mainFile3['Provider CCN'].count()

6094

In [60]:
#Remove hospitals that did not report revenue (Lose 273 to missing or unreported revenue events)
mainFile3 = mainFile3.loc[mainFile3['Net Patient Revenue'] >= 0.1]

In [61]:
#Quick Check
mainFile3['Provider CCN'].count()

5839

In [62]:
#Remove hospitals that discharged less than 15 patients (Lose 49 by placing a 15 patient minimum for study eligibility)
mainFile3 = mainFile3.loc[mainFile3['Total Discharges (V + XVIII + XIX + Unknown)'] >= 15]

In [63]:
#Quick Check
mainFile3['Provider CCN'].count()

5792

In [64]:
#Remove hospitals with less than 10 employees on payroll (Lose 23 by placing a 10 employee minimum for study eligibility)
mainFile3 = mainFile3.loc[mainFile3['FTE - Employees on Payroll'] >= 10]

In [65]:
#Quick Check
mainFile3['Provider CCN'].count()

5758

In [66]:
#Targets dataset on certificate of need states
#mainFile3 = mainFile3.loc[mainFile2['CON'] == 1]

In [67]:
#Remove net income outliers
#mainFile3 = mainFile3.loc[mainFile2['Net Income %'] <= .3304]

In [68]:
#Remove net income outliers
#mainFile3 = mainFile3.loc[mainFile2['Net Income %'] >= -.2473]

In [69]:
#Remove Discharges Per Bed outliers
#mainFile3 = mainFile3.loc[mainFile2['Rev Per Discharge'] <= 90000]

In [70]:
#Remove Rev Per Bed outliers
#mainFile3 = mainFile3.loc[mainFile2['Rev Per Bed'] <= 2000000]

In [71]:
#Remove Rev Per FTE outliers
#mainFile3 = mainFile3.loc[mainFile2['Rev Per FTE'] <= 500000]

In [72]:
#Remove Rev Per Discharge outliers
#mainFile3 = mainFile3.loc[mainFile2['Rev Per Discharge'] <= 175000]

In [73]:
#Targets dataset on STH locations (~56% of eligible continental US locations are STH providers)
#mainFile3 = mainFile3.loc[mainFile2['CCN Facility Type'] == 'STH']

In [74]:
#Quick Check
mainFile3['Provider CCN'].count()

5758

In [75]:
#Quick check on distribution of bronze, silver and gold
#mainFile3['Net Income Score'].value_counts()

In [76]:
#Quick Check
mainFile3['Provider CCN'].count()

5758

In [77]:
#mainFile3.to_csv("main3_2017.csv", index=False, header=True)

In [78]:
# ... Target file based on relevant features

In [79]:
mainFile4 = mainFile3

In [80]:
#Targeting the dataset on metrics important to Kent
mainFile4 = mainFile4[['Provider CCN','Hospital Name','Street Address','City','State Code','Zip Code', 'lat','lng','Rural Versus Urban', 'CCN Facility Type', 'Fiscal Year Begin Date', 'Fiscal Year End Date', 'FTE - Employees on Payroll', 'Number of Beds', 'Total Bed Days Available', 'Total Discharges (V + XVIII + XIX + Unknown)', 'Net Patient Revenue', 'Net Income', 'Wage-Related Costs (Core)', 'Total Salaries (adjusted)', 'Contract Labor', 'Rev Per FTE', 'CON','Urban','Inpatient %','Outpatient %','Charity %','Uncompensated %','Rev/Discharge','Rev Per Bed','Rev Per Bed Days', 'Discharges Per Bed','Discharges Per Bed Days','Discharge/FTE','Net Income %', 'Net Income Score', 'Labor %', 'Overhead %', 'Depreciation %', 'LbrExp/FTE', 'Overhead Exp Per Bed']]
mainFile4.head(2)


,Provider CCN,Hospital Name,Street Address,City,State Code,Zip Code,lat,lng,Rural Versus Urban,CCN Facility Type,...,Discharges Per Bed,Discharges Per Bed Days,Discharge/FTE,Net Income %,Net Income Score,Labor %,Overhead %,Depreciation %,LbrExp/FTE,Overhead Exp Per Bed
0,310050,SAINT CLARES HOSPITAL- DENVILLE,25 POCONO ROAD,DENVILLE,NJ,07834,40.894920,-74.465444,U,STH,...,12.495385,0.135819,12.495385,0.105576,2,0.436872,0.541184,0.034317,19215.838152,132486.553846
2,340119,STANLY REGIONAL MEDICAL CENTER,301 YADKIN STREET,ALBEMARLE,NC,28001,35.364497,-80.195004,U,STH,...,6.463918,0.107732,6.463918,-0.103473,3,0.594935,0.708396,0.107413,31006.724429,90054.536082


In [81]:
# Identify incomplete rows
mainFile4.count()

Provider CCN                                    5758
Hospital Name                                   5758
Street Address                                  5756
City                                            5758
State Code                                      5758
Zip Code                                        5758
lat                                             5676
lng                                             5676
Rural Versus Urban                              5758
CCN Facility Type                               5758
Fiscal Year Begin Date                          5758
Fiscal Year End Date                            5758
FTE - Employees on Payroll                      5758
Number of Beds                                  5757
Total Bed Days Available                        5758
Total Discharges (V + XVIII + XIX + Unknown)    5758
Net Patient Revenue                             5758
Net Income                                      5751
Wage-Related Costs (Core)                     

In [82]:
mainFile4 = mainFile4.dropna(how='any')

In [83]:
# Identify incomplete rows
mainFile4.count()

Provider CCN                                    2758
Hospital Name                                   2758
Street Address                                  2758
City                                            2758
State Code                                      2758
Zip Code                                        2758
lat                                             2758
lng                                             2758
Rural Versus Urban                              2758
CCN Facility Type                               2758
Fiscal Year Begin Date                          2758
Fiscal Year End Date                            2758
FTE - Employees on Payroll                      2758
Number of Beds                                  2758
Total Bed Days Available                        2758
Total Discharges (V + XVIII + XIX + Unknown)    2758
Net Patient Revenue                             2758
Net Income                                      2758
Wage-Related Costs (Core)                     

In [84]:
#Output targeted file
mainFile4.to_csv("main4_2016.csv", index=False, header=True)

In [108]:
# ... Net Income to feature review

In [109]:
targetFile5 = mainFile4

In [110]:
#Outlier Review
chartdata = targetFile5["Discharges Per Bed"]
quartiles = chartdata.quantile([.25,.5,.75])
lowerq = quartiles[0.25]
upperq = quartiles[0.75]
iqr = upperq-lowerq

print(f"The lower quartile of profit percent is: {lowerq}")
print(f"The upper quartile of profit percent is: {upperq}")
print(f"The interquartile range of profit percent is: {iqr}")
print(f"The the median of profit percent is: {quartiles[0.5]} ")

lower_bound = lowerq - (1.5*iqr)
upper_bound = upperq + (1.5*iqr)
print(f"Values below {lower_bound} could be outliers.")
print(f"Values above {upper_bound} could be outliers.")

The lower quartile of profit percent is: 32.25390625
The upper quartile of profit percent is: 54.6682518115942
The interquartile range of profit percent is: 22.414345561594203
The the median of profit percent is: 45.35625 
Values below -1.3676120923913047 could be outliers.
Values above 88.2897701539855 could be outliers.


In [111]:
#NI v. discharges per bed
# x_axis = targetFile5["Discharges Per Bed"]
# y_axis = targetFile5["Net Income %"]
# plt.title("Net Income % v. Discharges Per Bed")
# plt.xlabel("Discharges Per Bed")
# plt.ylabel("Net Income %")
# plt.scatter(x_axis, y_axis, marker="o", color="blue")
# (slope, intercept, rvalue, pvalue, stderr) = linregress(x_axis, y_axis)
# regress_values = x_axis * slope + intercept
# line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
# plt.plot(x_axis,regress_values,"r-")
# plt.annotate(line_eq,(0,50),fontsize=15,color="red")
# #plt.savefig("3bchart_days2unemployment_k.png")
# plt.show()
# print(f"The correlation coefficient is {round(st.pearsonr(x_axis,y_axis)[0],2)}")
# print(f"The line equation is as follows:  {line_eq}")

In [112]:
#NI v. discharges per bed days
# x_axis = targetFile5["Discharges Per Bed Days"]
# y_axis = targetFile5["Net Income %"]
# plt.title("Net Income % v. Discharges Per Bed Days")
# plt.xlabel("Discharges Per Bed Days")
# plt.ylabel("Net Income %")
# plt.scatter(x_axis, y_axis, marker="o", color="blue")
# #ax.set_yscale('log')
# (slope, intercept, rvalue, pvalue, stderr) = linregress(x_axis, y_axis)
# regress_values = x_axis * slope + intercept
# line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
# plt.plot(x_axis,regress_values,"r-")
# plt.annotate(line_eq,(0,50),fontsize=15,color="red")
# #plt.savefig("3bchart_days2unemployment_k.png")
# plt.show()
# print(f"The correlation coefficient is {round(st.pearsonr(x_axis,y_axis)[0],2)}")
# print(f"The line equation is as follows:  {line_eq}")

In [113]:
#Outlier Review
chartdata = targetFile5["Rev Per Bed"]
quartiles = chartdata.quantile([.25,.5,.75])
lowerq = quartiles[0.25]
upperq = quartiles[0.75]
iqr = upperq-lowerq

print(f"The lower quartile of profit percent is: {lowerq}")
print(f"The upper quartile of profit percent is: {upperq}")
print(f"The interquartile range of profit percent is: {iqr}")
print(f"The the median of profit percent is: {quartiles[0.5]} ")

lower_bound = lowerq - (1.5*iqr)
upper_bound = upperq + (1.5*iqr)
print(f"Values below {lower_bound} could be outliers.")
print(f"Values above {upper_bound} could be outliers.")

The lower quartile of profit percent is: 784693.5807268227
The upper quartile of profit percent is: 1527917.610272826
The interquartile range of profit percent is: 743224.0295460032
The the median of profit percent is: 1121975.4186046512 
Values below -330142.46359218203 could be outliers.
Values above 2642753.6545918304 could be outliers.


In [114]:
#NI v. Revenue Per Bed
# x_axis = targetFile5["Rev Per Bed"]
# y_axis = targetFile5["Net Income %"]
# plt.title("Net Income % v. Rev Per Bed")
# plt.xlabel("Rev Per Bed")
# plt.ylabel("Net Income %")
# plt.scatter(x_axis, y_axis, marker="o", color="blue")
# (slope, intercept, rvalue, pvalue, stderr) = linregress(x_axis, y_axis)
# regress_values = x_axis * slope + intercept
# line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
# plt.plot(x_axis,regress_values,"r-")
# plt.annotate(line_eq,(0,50),fontsize=15,color="red")
# #plt.savefig("3bchart_days2unemployment_k.png")
# plt.show()
# print(f"The correlation coefficient is {round(st.pearsonr(x_axis,y_axis)[0],2)}")
# print(f"The line equation is as follows:  {line_eq}")

In [117]:
#Outlier Review
chartdata = targetFile5["Rev/FTE"]
quartiles = chartdata.quantile([.25,.5,.75])
lowerq = quartiles[0.25]
upperq = quartiles[0.75]
iqr = upperq-lowerq

print(f"The lower quartile is: {lowerq}")
print(f"The upper quartile is: {upperq}")
print(f"The interquartile range is: {iqr}")
print(f"The the median is: {quartiles[0.5]} ")

lower_bound = lowerq - (1.5*iqr)
upper_bound = upperq + (1.5*iqr)
print(f"Values below {lower_bound} could be outliers.")
print(f"Values above {upper_bound} could be outliers.")

KeyError: 'Rev/FTE'

In [118]:
#NI v. Rev Per FTE
targetFile6 = targetFile5
# targetFile6 = targetFile5.loc[targetFile5['Rev Per FTE'] <= 1000000]
# targetFile6 = targetFile6.loc[targetFile5['Net Income %'] <= .35]
# targetFile6 = targetFile6.loc[targetFile5['Net Income %'] >= -.25]
# x_axis = targetFile6["Rev Per FTE"]
# y_axis = targetFile6["Net Income %"]
# plt.title("Net Income % v. Rev Per FTE")
# plt.xlabel("Rev Per FTE")
# plt.ylabel("Net Income %")
# plt.scatter(x_axis, y_axis, marker="o", color="blue")
# (slope, intercept, rvalue, pvalue, stderr) = linregress(x_axis, y_axis)
# regress_values = x_axis * slope + intercept
# line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
# plt.plot(x_axis,regress_values,"r-")
# plt.annotate(line_eq,(0,50),fontsize=15,color="red")
# #plt.savefig("3bchart_days2unemployment_k.png")
# plt.show()
# print(f"The correlation coefficient is {round(st.pearsonr(x_axis,y_axis)[0],2)}")
# print(f"The line equation is as follows:  {line_eq}")

In [120]:
#Outlier Review
chartdata = targetFile5["Rev/Discharge"]
quartiles = chartdata.quantile([.25,.5,.75])
lowerq = quartiles[0.25]
upperq = quartiles[0.75]
iqr = upperq-lowerq

print(f"The lower quartile of profit percent is: {lowerq}")
print(f"The upper quartile of profit percent is: {upperq}")
print(f"The interquartile range of profit percent is: {iqr}")
print(f"The the median of profit percent is: {quartiles[0.5]} ")

lower_bound = lowerq - (1.5*iqr)
upper_bound = upperq + (1.5*iqr)
print(f"Values below {lower_bound} could be outliers.")
print(f"Values above {upper_bound} could be outliers.")

The lower quartile of profit percent is: 19891.721485297996
The upper quartile of profit percent is: 33663.86064043294
The interquartile range of profit percent is: 13772.139155134944
The the median of profit percent is: 25105.884304789317 
Values below -766.4872474044205 could be outliers.
Values above 54322.06937313535 could be outliers.


In [121]:
#NI v. Rev Per Discharge
# x_axis = targetFile5["Rev Per Discharge"]
# y_axis = targetFile5["Net Income %"]
# plt.title("Net Income % v. Rev Per Discharge")
# plt.xlabel("Rev Per Discharge")
# plt.ylabel("Net Income %")
# plt.scatter(x_axis, y_axis, marker="o", color="blue")
# (slope, intercept, rvalue, pvalue, stderr) = linregress(x_axis, y_axis)
# regress_values = x_axis * slope + intercept
# line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
# plt.plot(x_axis,regress_values,"r-")
# plt.annotate(line_eq,(0,50),fontsize=15,color="red")
# #plt.savefig("3bchart_days2unemployment_k.png")
# plt.show()
# print(f"The correlation coefficient is {round(st.pearsonr(x_axis,y_axis)[0],2)}")
# print(f"The line equation is as follows:  {line_eq}")

In [123]:
#Outlier Review
chartdata = targetFile5["LbrExp FTE"]
quartiles = chartdata.quantile([.25,.5,.75])
lowerq = quartiles[0.25]
upperq = quartiles[0.75]
iqr = upperq-lowerq

print(f"The lower quartile is: {lowerq}")
print(f"The upper quartile is: {upperq}")
print(f"The interquartile range is: {iqr}")
print(f"The the median of is: {quartiles[0.5]} ")

lower_bound = lowerq - (1.5*iqr)
upper_bound = upperq + (1.5*iqr)
print(f"Values below {lower_bound} could be outliers.")
print(f"Values above {upper_bound} could be outliers.")

KeyError: 'LbrExp FTE'

In [124]:
#NI v. Labor Expense Per FTE
# targetFile6 = targetFile5.loc[targetFile5['Labor Exp Per FTE'] <= 130000]
# targetFile6 = targetFile6.loc[targetFile5['Net Income %'] <= .35]
# targetFile6 = targetFile6.loc[targetFile5['Net Income %'] >= -.25]
# x_axis = targetFile6["Labor Exp Per FTE"]
# y_axis = targetFile6["Net Income %"]
# plt.title("Net Income % v. Labor Exp Per FTE")
# plt.xlabel("Labor Exp Per FTE")
# plt.ylabel("Net Income %")
# plt.scatter(x_axis, y_axis, marker="o", color="blue")
# (slope, intercept, rvalue, pvalue, stderr) = linregress(x_axis, y_axis)
# regress_values = x_axis * slope + intercept
# line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
# plt.plot(x_axis,regress_values,"r-")
# plt.annotate(line_eq,(0,50),fontsize=15,color="red")
# #plt.savefig("3bchart_days2unemployment_k.png")
# plt.show()
# print(f"The correlation coefficient is {round(st.pearsonr(x_axis,y_axis)[0],2)}")
# print(f"The line equation is as follows:  {line_eq}")

In [125]:
#Outlier Review
chartdata = targetFile5["Overhead %"]
quartiles = chartdata.quantile([.25,.5,.75])
lowerq = quartiles[0.25]
upperq = quartiles[0.75]
iqr = upperq-lowerq

print(f"The lower quartile is: {lowerq}")
print(f"The upper quartile is: {upperq}")
print(f"The interquartile range is: {iqr}")
print(f"The the median of is: {quartiles[0.5]} ")

lower_bound = lowerq - (1.5*iqr)
upper_bound = upperq + (1.5*iqr)
print(f"Values below {lower_bound} could be outliers.")
print(f"Values above {upper_bound} could be outliers.")

The lower quartile is: 0.5601867271196768
The upper quartile is: 0.683142643808776
The interquartile range is: 0.12295591668909922
The the median of is: 0.6157974834855593 
Values below 0.37575285208602793 could be outliers.
Values above 0.8675765188424248 could be outliers.


In [126]:
#NI v. Overhead Percent
# targetFile6 = targetFile5.loc[targetFile5['Overhead %'] <= .90]
# targetFile6 = targetFile6.loc[targetFile5['Net Income %'] <= .35]
# targetFile6 = targetFile6.loc[targetFile5['Net Income %'] >= -.25]
# x_axis = targetFile6["Overhead %"]
# y_axis = targetFile6["Net Income %"]
# plt.title("Net Income % v. Overhead %")
# plt.xlabel("Overhead %")
# plt.ylabel("Net Income %")
# plt.scatter(x_axis, y_axis, marker="o", color="blue")
# (slope, intercept, rvalue, pvalue, stderr) = linregress(x_axis, y_axis)
# regress_values = x_axis * slope + intercept
# line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
# plt.plot(x_axis,regress_values,"r-")
# plt.annotate(line_eq,(0,50),fontsize=15,color="red")
# #plt.savefig("3bchart_days2unemployment_k.png")
# plt.show()
# print(f"The correlation coefficient is {round(st.pearsonr(x_axis,y_axis)[0],2)}")
# print(f"The line equation is as follows:  {line_eq}")

In [127]:
#Outlier Review
chartdata = targetFile5["Labor %"]
quartiles = chartdata.quantile([.25,.5,.75])
lowerq = quartiles[0.25]
upperq = quartiles[0.75]
iqr = upperq-lowerq

print(f"The lower quartile is: {lowerq}")
print(f"The upper quartile is: {upperq}")
print(f"The interquartile range is: {iqr}")
print(f"The the median of is: {quartiles[0.5]} ")

lower_bound = lowerq - (1.5*iqr)
upper_bound = upperq + (1.5*iqr)
print(f"Values below {lower_bound} could be outliers.")
print(f"Values above {upper_bound} could be outliers.")

The lower quartile is: 0.37626185500721265
The upper quartile is: 0.5315353561193963
The interquartile range is: 0.15527350111218363
The the median of is: 0.4499461703678241 
Values below 0.1433516033389372 could be outliers.
Values above 0.7644456077876718 could be outliers.


In [128]:
#NI v. Labor Percent
targetFile6 = targetFile5
# targetFile6 = targetFile5.loc[targetFile5['Labor %'] <= .90]
# targetFile6 = targetFile6.loc[targetFile5['Net Income %'] <= .35]
# targetFile6 = targetFile6.loc[targetFile5['Net Income %'] >= -.25]
# x_axis = targetFile6["Labor %"]
# y_axis = targetFile6["Net Income %"]
# plt.title("Net Income % v. Labor %")
# plt.xlabel("Labor %")
# plt.ylabel("Net Income %")
# plt.scatter(x_axis, y_axis, marker="o", color="blue")
# (slope, intercept, rvalue, pvalue, stderr) = linregress(x_axis, y_axis)
# regress_values = x_axis * slope + intercept
# line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
# plt.plot(x_axis,regress_values,"r-")
# plt.annotate(line_eq,(0,50),fontsize=15,color="red")
# #plt.savefig("3bchart_days2unemployment_k.png")
# plt.show()
# print(f"The correlation coefficient is {round(st.pearsonr(x_axis,y_axis)[0],2)}")
# print(f"The line equation is as follows:  {line_eq}")

In [132]:
#Outlier Review
chartdata = targetFile5["Discharge/FTE"]
quartiles = chartdata.quantile([.25,.5,.75])
lowerq = quartiles[0.25]
upperq = quartiles[0.75]
iqr = upperq-lowerq

print(f"The lower quartile is: {lowerq}")
print(f"The upper quartile is: {upperq}")
print(f"The interquartile range is: {iqr}")
print(f"The the median of is: {quartiles[0.5]} ")

lower_bound = lowerq - (1.5*iqr)
upper_bound = upperq + (1.5*iqr)
print(f"Values below {lower_bound} could be outliers.")
print(f"Values above {upper_bound} could be outliers.")

The lower quartile is: 32.25390625
The upper quartile is: 54.6682518115942
The interquartile range is: 22.414345561594203
The the median of is: 45.35625 
Values below -1.3676120923913047 could be outliers.
Values above 88.2897701539855 could be outliers.


In [133]:
#NI v. Labor Percent
# targetFile6 = targetFile5
# targetFile6 = targetFile5.loc[targetFile5['Discharges Per FTE'] <= 120]
# targetFile6 = targetFile6.loc[targetFile5['Net Income %'] <= .35]
# targetFile6 = targetFile6.loc[targetFile5['Net Income %'] >= -.25]
# x_axis = targetFile6["Discharges Per FTE"]
# y_axis = targetFile6["Net Income %"]
# plt.title("Net Income % v. Discharges Per FTE")
# plt.xlabel("Discharges Per FTE")
# plt.ylabel("Net Income %")
# plt.scatter(x_axis, y_axis, marker="o", color="blue")
# (slope, intercept, rvalue, pvalue, stderr) = linregress(x_axis, y_axis)
# regress_values = x_axis * slope + intercept
# line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
# plt.plot(x_axis,regress_values,"r-")
# plt.annotate(line_eq,(0,50),fontsize=15,color="red")
# #plt.savefig("3bchart_days2unemployment_k.png")
# plt.show()
# print(f"The correlation coefficient is {round(st.pearsonr(x_axis,y_axis)[0],2)}")
# print(f"The line equation is as follows:  {line_eq}")

In [136]:
#Start machine learning ...

In [137]:
# Select features to be used as x values
features1 = mainFile4[['CON','Urban','Inpatient %','Outpatient %','Charity %','Uncompensated %','Rev/Discharge','Rev Per Bed','Rev Per Bed Days', 'Discharges Per Bed','Discharges Per Bed Days','Discharge/FTE', 'Labor %', 'Overhead %', 'Depreciation %', 'LbrExp/FTE', 'Overhead Exp Per Bed']]
features1.head(2)
#RFE logrithmic output ... array([15, 17, 16, 14, 12,  8, 11,  6,  5 (R/BDay),  7,  9,  3(D/FTE),  2(L%),  1 (OH%),  4 (D%), 13, 10])

,CON,Urban,Inpatient %,Outpatient %,Charity %,Uncompensated %,Rev/Discharge,Rev Per Bed,Rev Per Bed Days,Discharges Per Bed,Discharges Per Bed Days,Discharge/FTE,Labor %,Overhead %,Depreciation %,LbrExp/FTE,Overhead Exp Per Bed
0,1.0,1.0,0.609156,0.390844,0.018000,0.018000,19591.934499,244808.756923,2660.964749,12.495385,0.135819,12.495385,0.436872,0.541184,0.034317,19215.838152,132486.553846
2,1.0,1.0,0.312861,0.687139,0.008871,0.008871,19666.786284,127124.484536,2118.741409,6.463918,0.107732,6.463918,0.594935,0.708396,0.107413,31006.724429,90054.536082


In [138]:
#Output features file as a reference document
features1.to_csv("features2016.csv", index=False, header=True)

In [139]:
# ... Linear Regression Model

In [140]:
# Identify the outcome column/data
y = mainFile4[['Net Income %']]
y.head()

,Net Income %
0,0.105576
2,-0.103473
4,0.255633
5,-0.094758
6,-0.021945


In [141]:
mainFile4["Net Income %"].dtype

dtype('float64')

In [142]:
y.shape

(3035, 1)

In [143]:
X = features1
X.shape

(3035, 17)

In [144]:
#Create a training and testing dataset - one hot
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=1)

In [145]:
y_train.shape

(2276, 1)

In [146]:
X_train.shape

(2276, 17)

In [147]:
y_test.shape

(759, 1)

In [148]:
X_test.shape

(759, 17)

In [149]:
# Scale your data
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [150]:
#Create the linear regression model and fit to the training data
#score is r2 root mean ... 1 is better
modela = LinearRegression()
modela.fit(X_train_scaled, y_train)
score = modela.score(X_train_scaled, y_train)
print(f"Training Data Score: {modela.score(X_train_scaled, y_train)}")

Training Data Score: 0.4323507515031151


In [151]:
#Run the model with the testing data and determine score
print(f"Testing Data Score: {modela.score(X_test_scaled, y_test)}")

Testing Data Score: 0.5165191828765643


In [152]:
# ... Linear Recursive Feature Elimination

In [153]:
#Identify features to create a more efficient model with
estimator = LinearRegression()
selector = RFE(estimator, n_features_to_select=5, step=1)
selector = selector.fit(X_train_scaled, y_train)

In [154]:
selector.support_

array([False, False, False, False, False,  True, False, False,  True,
       False,  True, False,  True,  True, False, False, False])

In [155]:
selector.ranking_

array([11, 13,  9, 12,  6,  1,  4,  8,  1,  5,  1,  2,  1,  1,  7, 10,  3])

In [156]:
# ... Logistics Model

In [157]:
# Identify the outcome column/data
yy = mainFile4[['Net Income Score']]
yy.head()

,Net Income Score
0,2
2,3
4,1
5,3
6,2


In [158]:
yy.shape

(3035, 1)

In [159]:
XX = features1
XX.shape

(3035, 17)

In [160]:
#Create a training and testing dataset - one hot
X_train, X_test, y_train, y_test = train_test_split(
    XX, yy, random_state=1)

In [161]:
y_train.shape

(2276, 1)

In [162]:
X_train.shape

(2276, 17)

In [163]:
y_test.shape

(759, 1)

In [164]:
y_test.shape

(759, 1)

In [165]:
# Scale your data
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [166]:
#Create the logistic regression model and fit to the training data
#score is r2 root mean ... 1 is better
modela = LogisticRegression()
modela.fit(X_train_scaled, y_train)
score = modela.score(X_train_scaled, y_train)
print(f"Training Data Score: {modela.score(X_train_scaled, y_train)}")

Training Data Score: 0.6542179261862917


C:\Users\rodge\anaconda3\envs\PythonData\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\rodge\anaconda3\envs\PythonData\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [167]:
#Run the model with the testing data and determine score
print(f"Testing Data Score: {modela.score(X_test_scaled, y_test)}")

Testing Data Score: 0.6666666666666666


In [168]:
# ... recursive feature elimination

In [169]:
#Identify features to create a more efficient model with
estimator = LogisticRegression()
selector = RFE(estimator, n_features_to_select=1, step=1)
selector = selector.fit(X_train_scaled, y_train)

C:\Users\rodge\anaconda3\envs\PythonData\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\rodge\anaconda3\envs\PythonData\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
C:\Users\rodge\anaconda3\envs\PythonData\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Pleas

In [170]:
selector.support_

array([False, False, False, False, False, False, False, False, False,
       False, False, False,  True, False, False, False, False])

In [171]:
selector.ranking_

array([12, 16, 14, 17,  9,  6, 11,  5, 13,  4, 15,  3,  1,  2,  7, 10,  8])

In [ ]:
#End ... kent matthew rodgers, profitableHospital-project